In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!pip install -U tensorflow-addons
!pip install --quiet vit-keras

In [ ]:
import os
import matplotlib.pyplot as plt
import keras as ks
from keras import layers
from keras.models import Sequential
import tensorflow as tf
import glob,warnings 
warnings.filterwarnings('ignore')

import pandas as pd 
import numpy as np 
import os 
import tensorflow as tf
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten
from tensorflow.keras.models import Sequential, Model
import matplotlib.pyplot as plt 
import seaborn as sns
import matplotlib.image as mpimg
%matplotlib inline

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

from sklearn.metrics import confusion_matrix, classification_report




In [ ]:
#IMG_WIDTH = 72
#IMG_HEIGHT = 72

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
num_epochs = 100
image_size = 128  # We'll resize input images to this size
image_size_read = (128,128)
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [ ]:

## try from here
train_directory = '/content/drive/MyDrive/Colab Notebooks/BT5153 Applied ML Project /5 classes 5k each for train/Training'
val_directory = '/content/drive/MyDrive/Colab Notebooks/BT5153 Applied ML Project /5 classes 5k each for train/Validation'
test_directory = '/content/drive/MyDrive/Colab Notebooks/BT5153 Applied ML Project /5 classes 5k each for train/Testing'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_directory, label_mode='categorical', image_size=image_size_read, batch_size=batch_size)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(val_directory, label_mode='categorical', image_size=image_size_read, batch_size=batch_size)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(test_directory, label_mode='categorical', image_size=image_size_read, batch_size=batch_size)

#train_ds = train_ds.prefetch(buffer_size=128)
#val_ds = val_ds.prefetch(buffer_size=128)
#test_ds = test_ds.prefetch(buffer_size=128)

Found 25000 files belonging to 5 classes.
Found 27447 files belonging to 5 classes.
Found 34281 files belonging to 5 classes.


In [ ]:
#data_augmentation = keras.Sequential(
#    [
#        layers.Normalization(),
#        layers.Resizing(image_size, image_size),
#        layers.RandomFlip("horizontal"),
#        layers.RandomRotation(factor=0.02),
#        layers.RandomZoom(
#            height_factor=0.2, width_factor=0.2
#        ),
#    ],
#    name="data_augmentation",
#)
# Compute the mean and the variance of the training data for normalization.
#data_augmentation.layers[0].adapt(X_train)


## Build the model

In [ ]:
from vit_keras import vit

### ViT B32 Model

In [ ]:
vit_model = vit.vit_b32(
        image_size = image_size,
        activation = 'softmax',
        pretrained = True,
        include_top = False,
        pretrained_top = False, #do not include the classifier at the top 
        classes = 5) #5 class for ours 

In [ ]:
# Freeze the base_model
#vit_model.trainable = False

#### Fine-tune

In [ ]:
model = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(11, activation = tfa.activations.gelu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(5, 'softmax') #5 classes
    ],
    name = 'vision_transformer')

model.summary()

Model: "vision_transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit-b32 (Functional)        (None, 768)               87429888  
                                                                 
 flatten_1 (Flatten)         (None, 768)               0         
                                                                 
 batch_normalization_2 (Batc  (None, 768)              3072      
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 11)                8459      
                                                                 
 batch_normalization_3 (Batc  (None, 11)               44        
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 5)          

In [ ]:
optimizer = tfa.optimizers.RectifiedAdam(learning_rate = learning_rate)

model.compile(optimizer = optimizer, 
              loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics = ['accuracy',
                         tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5,name='mcc')])

In [ ]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                                 patience = 10,
                                                 mode = 'min',
                                                 restore_best_weights = True,
                                                 verbose = 1)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                                                 factor = 0.3,
                                                 patience = 3,
                                                 verbose = 1,
                                                 min_delta = 1e-4,
                                                 mode = 'min')

checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/5153 weights/batchsize32_ViT_B-32_5C5K_{epoch}.h5' #/tmp/checkpoint #
checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_loss",
        save_best_only=True,
       # save_weights_only=True, #don't save weights, save model 
    )

callbacks = [earlystopping, reduce_lr, checkpoint_callback]

In [ ]:
model.metrics_names

[]

In [ ]:
model.fit(
        train_ds,
        #x=X_train,
        #y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=val_ds,
       # steps_per_epoch = 328,
       # validation_steps= 431,
        #validation_split=0.1,
        verbose=1,
        callbacks = callbacks)



Epoch 1/100
782/782 [==============================] - 239s 273ms/step - loss: 1.4082 - accuracy: 0.4071 - mcc: 0.2601 - val_loss: 1.8320 - val_accuracy: 0.2681 - val_mcc: 0.1193 - lr: 0.0010
Epoch 2/100
782/782 [==============================] - 207s 265ms/step - loss: 1.1717 - accuracy: 0.5255 - mcc: 0.4079 - val_loss: 2.1102 - val_accuracy: 0.1831 - val_mcc: 0.0804 - lr: 0.0010
Epoch 3/100
782/782 [==============================] - 211s 270ms/step - loss: 1.1004 - accuracy: 0.5603 - mcc: 0.4509 - val_loss: 1.7493 - val_accuracy: 0.2519 - val_mcc: 0.1367 - lr: 0.0010
Epoch 4/100
782/782 [==============================] - 206s 264ms/step - loss: 1.0554 - accuracy: 0.5787 - mcc: 0.4738 - val_loss: 2.3820 - val_accuracy: 0.3460 - val_mcc: 0.2486 - lr: 0.0010
Epoch 5/100
782/782 [==============================] - 206s 263ms/step - loss: 1.0381 - accuracy: 0.5870 - mcc: 0.4840 - val_loss: 2.4534 - val_accuracy: 0.4078 - val_mcc: 0.1231 - lr: 0.0010
Epoch 6/100
782/782 [===================

In [ ]:
test_loss, test_accuracy, test_mcc = model.evaluate(test_ds)

1072/1072 [==============================] - 72s 67ms/step - loss: 1.1001 - accuracy: 0.5789 - mcc: 0.4308


In [ ]:
print(f'Accuracy is {round(test_accuracy * 100, 2)}%')

Accuracy is 57.89%


In [ ]:
true = []
pred = []
batches = int(np.ceil(34281/32)) #34281 -- > test size / 32  is batch size 
i = 1
for image, label in test_ds:
  pred = np.concatenate([pred, 
                         np.argmax(model.predict(image), axis=-1)])
  true = np.concatenate([true,
                         np.argmax(label.numpy(), axis=-1)])
  print('{}/{}'.format(i, batches))
  i += 1

1/1072
2/1072
3/1072
4/1072
5/1072
6/1072
7/1072
8/1072
9/1072
10/1072
11/1072
12/1072
13/1072
14/1072
15/1072
16/1072
17/1072
18/1072
19/1072
20/1072
21/1072
22/1072
23/1072
24/1072
25/1072
26/1072
27/1072
28/1072
29/1072
30/1072
31/1072
32/1072
33/1072
34/1072
35/1072
36/1072
37/1072
38/1072
39/1072
40/1072
41/1072
42/1072
43/1072
44/1072
45/1072
46/1072
47/1072
48/1072
49/1072
50/1072
51/1072
52/1072
53/1072
54/1072
55/1072
56/1072
57/1072
58/1072
59/1072
60/1072
61/1072
62/1072
63/1072
64/1072
65/1072
66/1072
67/1072
68/1072
69/1072
70/1072
71/1072
72/1072
73/1072
74/1072
75/1072
76/1072
77/1072
78/1072
79/1072
80/1072
81/1072
82/1072
83/1072
84/1072
85/1072
86/1072
87/1072
88/1072
89/1072
90/1072
91/1072
92/1072
93/1072
94/1072
95/1072
96/1072
97/1072
98/1072
99/1072
100/1072
101/1072
102/1072
103/1072
104/1072
105/1072
106/1072
107/1072
108/1072
109/1072
110/1072
111/1072
112/1072
113/1072
114/1072
115/1072
116/1072
117/1072
118/1072
119/1072
120/1072
121/1072
122/1072
123/1072
1

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(true, pred)

0.5789212683410636

In [ ]:
df = pd.DataFrame({'true':true,
                   'pred':pred})

df.to_csv('/content/drive/MyDrive/Colab Notebooks/5153 weights/batchsize32_ViT_pretrained_test_pred.csv')